In [1]:
import pandas as pd
import missingno as msno
from matplotlib.pyplot import plot
import seaborn as sns
import matplotlib.pyplot as plt 

In [2]:
data = pd.read_csv('./event-data-from-20230608-to-20240606.csv')
data_active_users = pd.read_csv('./active-users.csv')

# Utils

In [3]:
def remove_columns_with_no_data(df: pd.DataFrame, coef_to_drop = 1.0):
    columns = df.columns
    dropped_columns = []
    for col in columns:
        per_of_missing_data = df[col].isna().sum() / len(df)
        if per_of_missing_data == coef_to_drop:
            dropped_columns.append(col)
            df = df.drop(columns=[col])
    return df, dropped_columns

def remove_not_useful_columns(df: pd.DataFrame, columns: list[str]):
    df = df.copy()
    return df.drop(columns=columns)

def convert_to_datetime_format(df, columns: list, format='%Y-%m-%d %H:%M'):
    df = df.copy()  # Make a copy to avoid SettingWithCopyWarning
    for col in columns:
        df[col] = pd.to_datetime(df[col], format=format)
    return df



# Archived Code

In [4]:
# ================== Find the pairs of the timestamp ===================

# groups = []
# ommit_indexes = []

# for index, value_df in sorted_sample.iterrows():
#     if index in ommit_indexes:
#         continue
#     indexes = [index]
#     for sub_index, sub_value_df in sorted_sample.iterrows():
#         if sub_index != index and sub_index not in ommit_indexes:
#             if sub_value_df['Start Date & Time'] == value_df['Start Date & Time']:
#                 indexes.append(sub_index)
#     if len(indexes) > 1:
#         groups.append(indexes)
#         ommit_indexes.extend(indexes)

# EDA

In [5]:
processed_data, dropped_colummns = remove_columns_with_no_data(data)
processed_data = remove_not_useful_columns(processed_data, ['Event UUID', 'Invitee UUID', 'Invitee accepted marketing emails', 
                                                            'Invitee Last Name', 'User Name', 'Question 1', 'Response 1', 
                                                            'Marked as No-Show'])
print(f"Dropped columns : {dropped_colummns}")
# msno.matrix(processed_data)

Dropped columns : ['Team', 'UTM Campaign', 'UTM Source', 'UTM Medium', 'UTM Term', 'UTM Content', 'Salesforce UUID', 'Event Price', 'Payment Currency', 'Guest Email(s)', 'Invitee Reconfirmed', 'Meeting Notes', 'Scheduling method']


In [6]:
processed_data['Start Date & Time'] = pd.to_datetime(processed_data['Start Date & Time'],  format='%Y-%m-%d %H:%M')
processed_data['End Date & Time'] = pd.to_datetime(processed_data['End Date & Time'],  format='%Y-%m-%d %H:%M')
processed_data['Event Created Date & Time'] = pd.to_datetime(processed_data['Event Created Date & Time'],  format='%Y-%m-%d %H:%M')

In [7]:
processed_data[['Invitee Email']].head(20)

,Invitee Email
0,mansonderek@hotmail.com
1,kabusha@tut.by
2,natakhudzinskaya@gmail.com
3,nastasiakarabelnik@gmail.com
4,anna.pekshueva@gmail.com
5,daria.poznyak@gmail.com
6,stasia962022@gmail.com
7,mansonderek@hotmail.com
8,dasha.grishanovich@gmail.com
9,malibu123098@gmail.com


# User's features

### Check the user's life period
1. Group the users by email 
2. 

In [8]:
import tqdm
def process_group(sample, features = {}):
    sample = convert_to_datetime_format(sample, ['Start Date & Time', 'End Date & Time', 'Event Created Date & Time'])
    sorted_sample = sample.sort_values(by='Start Date & Time')
    
    
    # Total user's life in days
    features["total_user_life_in_days"] = (sorted_sample['Start Date & Time'].iloc[-1] - sorted_sample['Start Date & Time'].iloc[0]).days

    # Total classes
    features["total_classes"] = sorted_sample.shape[0]

    # Total number of participation in exercices (not cancelled)
    features["total_presence"] = sorted_sample[sorted_sample['Canceled'] != True].shape[0]
                                  
    print(f"Features: {features}")

    # Total number of cancalled events
    features["total_cancelled"] = sorted_sample[sorted_sample['Canceled']].shape[0]

    # Cancelled / Total Ratio
    features["cancel_total_ratio"] = features["total_cancelled"] / features["total_classes"]

    # Presence / Total Ratio
    features["presence_total_ratio"] =  features["total_presence"] / features["total_classes"]

    # Participation per whole user's life Ratio
    features["participation_per_life"] = features["total_classes"] / features["total_user_life_in_days"]

    
    # Convert to pandas Dataframe
    features_df = pd.DataFrame([features])
    return features_df


def iterate_by_group(df: pd.DataFrame):
    df = df.copy()
    emails_list = df['Invitee Email'].unique()
    for email in tqdm.tqdm(emails_list):
        sample = df[df['Invitee Email'] == email]
        processed_group = process_group(sample)
        break # break after one sample ; for dev purposes    
    return df, sample, processed_group

In [9]:
_, sample, process_group = iterate_by_group(processed_data)

  0%|          | 0/661 [00:00<?, ?it/s]

Features: {'total_user_life_in_days': 158, 'total_classes': 36, 'total_presence': 20}


process_group

In [10]:
client = processed_data[processed_data['Invitee Email'] == 'margarita.minaeva.by@gmail.com']

In [11]:
sample = client.copy()
sample = convert_to_datetime_format(sample, ['Start Date & Time', 'End Date & Time', 'Event Created Date & Time'])
sorted_sample = sample.sort_values(by='Event Created Date & Time')

In [12]:
# Group by (day, training type) -> get cancel status -> list[bool]

sorted_sample[['Canceled', 'Start Date & Time', 'Canceled By', 'Cancellation reason', 'Event Type Name', 'Event Created Date & Time']]
filter_fn = lambda x: sorted_sample.loc[x]['Start Date & Time'].date()
grouped_sample = sorted_sample.groupby([filter_fn, 'Event Type Name'])
# grouped_sample.get_group((pd.to_datetime('2023-06-21').date(), 'Stretching'))['Canceled']
    

In [13]:

# Find metrics for training 'dat' vector (day, training type)

def find_pairs(seq, buffor):
    if not ((False in seq) and (True in seq)):
        resch = len(buffor) // 2
        return resch, seq

    i = 0
    while i < len(seq) - 1:
        if seq[i] != seq[i + 1]:
            buffor.append(seq[i])
            buffor.append(seq[i + 1])
            seq.pop(i)
            seq.pop(i)
            # After popping, continue without incrementing i
            continue
        i += 1
    return find_pairs(seq, buffor=buffor)
        


def get_metrics_from_training_group(seq: list[bool]):
    compl, cancel = 0, 0
    resch, seq = find_pairs(seq, [])
    if seq:
        if len(seq) == sum(seq):
            cancel += len(seq)
        else: 
            compl += len(seq)
        return (compl, resch, cancel)
    
    return (compl, resch, cancel)



overviews = [
    [False, True, True], 
    [False, False, True, True], 
    [False, False, False, True, True], 
    [True, True],                   
    [False, False],                 
    [True, False],                  
    [True],                         
    [False],                         
]

# compl, resch, cancel
grand_truth = [
    (0, 1, 1),
    (0, 2, 0),
    (1, 2, 0),
    (0, 0, 2),
    (2, 0, 0),
    (0, 1, 0),
    (0, 0, 1),
    (1, 0, 0)

]
def test_metric_pipeline(overviews, grand_truth):
    if len(overviews) != len(grand_truth):
        print(f'Lists have the differente length')
        return
    
    for example, gt in zip(overviews, grand_truth):
        result = get_metrics_from_training_group(example)
        if result == gt:
            print('Test passed!')
        else:
            print('Test not passed!')

test_metric_pipeline(overviews=overviews, grand_truth=grand_truth)

Test passed!
Test passed!
Test passed!
Test passed!
Test passed!
Test passed!
Test passed!
Test passed!


In [14]:
sample['Start Date & Time'] = pd.to_datetime(sample['Start Date & Time'],  format='%Y-%m-%d %H:%M')
sample['End Date & Time'] = pd.to_datetime(sample['End Date & Time'],  format='%Y-%m-%d %H:%M')
sample['Event Created Date & Time'] = pd.to_datetime(sample['Event Created Date & Time'],  format='%Y-%m-%d %H:%M')

In [15]:
sorted_sample = sample.sort_values(by='Event Created Date & Time')
(sorted_sample['Start Date & Time'].iloc[0] - sorted_sample['Start Date & Time'].iloc[-1]).days

-357

In [16]:
# TEST CASES 
import pandas as pd
case_1 = pd.DataFrame({
    "Start Date & Time": [pd.to_datetime('2023-06-21 20:05:00'),
                          pd.to_datetime('2023-06-21 19:00:00'),
                          pd.to_datetime('2023-06-21 20:05:00'),
                          ],
    "Event Created Data & Time": [pd.to_datetime('2023-06-18 02:20:00'),
                                  pd.to_datetime('2023-06-18 02:20:00'),
                                  pd.to_datetime('2023-06-21 16:28:00'),
                                  ], 
    "Canceled": [True, True, False],
    "Event Type Name": ['Stretching','Stretching','Stretching'],
    "Status": ['Rescheduled', 'Rescheduled', 'Completed']
    })

# True True False --> Resch Resch Compl

case_2 = pd.DataFrame({
    "Start Date & Time": [pd.to_datetime('2023-06-19 19:00:00'),
                          pd.to_datetime('2023-06-19 20:05:00'),
                          ],
    "Event Created Data & Time": [pd.to_datetime('2023-06-16 10:16:00'),
                                  pd.to_datetime('2023-06-16 10:17:00'),
                                  ], 
    "Canceled": [False, False],
    "Event Type Name": ['Stretching','Stretching'],
    "Status": ['Completed', 'Completed']
    })


case_3 = pd.DataFrame({
    "Start Date & Time": [pd.to_datetime('2023-06-22 18:00:00'),
                          ],
    "Event Created Data & Time": [pd.to_datetime('2023-06-18 02:21:00')
                                  ], 
    "Canceled": [True],
    "Event Type Name": ['Yoga'],
    "Status": ['Canceled']
    })


case_4 = pd.DataFrame({
    "Start Date & Time": [pd.to_datetime('2023-07-03 19:00:00'),
                          pd.to_datetime('2023-07-03 20:05:00'),
                          ],
    "Event Created Data & Time": [pd.to_datetime('2023-07-02 03:13:00'),
                                  pd.to_datetime('2023-07-02 03:47:00')
                                  ], 
    "Canceled": [True, False],
    "Event Type Name": ['Stretching', 'Stretching'],
    "Status": ['Reschudeled', 'Completed']
    })



case_5 = pd.DataFrame({
    "Start Date & Time": [pd.to_datetime('2023-10-31 20:05:00'), 
                          pd.to_datetime('2023-10-31 20:05:00')],
    "Event Created Data & Time": [pd.to_datetime('2023-10-26 11:12:00'),
                                  pd.to_datetime('2023-10-31 06:50:00')
                                  ], 
    "Canceled": [True, False],
    "Event Type Name": ['Stretching', 'Stretching'],
    "Status": ['Reschudeled', 'Completed']
})

case_6 = pd.DataFrame({
    "Start Date & Time": [pd.to_datetime('2023-11-15 17:00:00'), 
                          pd.to_datetime('2023-11-15 18:00:00')],
    "Event Created Data & Time": [pd.to_datetime('2023-11-15 10:29:00'),
                                  pd.to_datetime('2023-11-15 16:17:00')
                                  ], 
    "Canceled": [False, False],
    "Event Type Name": ['Stretching', 'Stretching'],
    "Status": ['Completed', 'Completed']
})



case_7 = pd.DataFrame({
    "Start Date & Time": [pd.to_datetime('2023-11-15 17:00:00'), 
                          pd.to_datetime('2023-11-15 18:00:00')],
    "Event Created Data & Time": [pd.to_datetime('2023-11-15 10:29:00'),
                                  pd.to_datetime('2023-11-15 16:17:00')
                                  ], 
    "Canceled": [True, True],
    "Event Type Name": ['Stretching', 'Stretching'],
    "Status": ['Reschudeled', 'Canceled']
})



case_8 = pd.DataFrame({
    "Start Date & Time": [pd.to_datetime('2024-02-18 11:00:00'), 
                          pd.to_datetime('2024-02-18 11:00:00')],
    "Event Created Data & Time": [pd.to_datetime('2024-02-16 12:29:00'),
                                  pd.to_datetime('2024-02-16 12:32:00')
                                  ], 
    "Canceled": [False, True],
    "Event Type Name": ['Stretching', 'Stretching'],
    "Status": ['Reschudeled', 'Canceled']
})

# True True False --> Resch Resch Compl
# True False False --> Resch Compl Compl
# False True True --> Resch Resch Compl
# False False --> Compl Compl
# True True --> Resch Cancel
# True --> Cancel
# False --> Compl
# True False --> Resch Compl
# False True --> Resch Cancel

# The event is completede if AT THE END OF DAY (for this event), the event won't reschedulled or canceled
# The event is cancelled if AT THE END OF DAY (for this event), the event won't completed

# If len(seq) > 1, check the last record:
#       if False (event completed), t

case_8


,Start Date & Time,Event Created Data & Time,Canceled,Event Type Name,Status
0,2024-02-18 11:00:00,2024-02-16 12:29:00,False,Stretching,Reschudeled
1,2024-02-18 11:00:00,2024-02-16 12:32:00,True,Stretching,Canceled


In [17]:
def process_sequence(sequence):
    # Initialize an empty list to hold the output labels
    labels = []

    # Loop through the sequence to determine the output labels based on the rules
    for i in range(len(sequence)):
        if sequence[i] == True:
            if i == len(sequence) - 1:  # If it's the last element
                labels.append("Cancel")
            else:
                labels.append("Resch")
        elif sequence[i] == False:
            if i == 0 or (i > 0 and sequence[i-1] == False):
                labels.append("Compl")
            else:
                labels.append("Compl")
        
        # Special condition for pairs to determine 'Cancel' or 'Compl'
        if i > 0 and sequence[i] == False and sequence[i-1] == True:
            if i == len(sequence) - 1:
                labels[-2] = "Resch"
                labels[-1] = "Compl"
            else:
                labels[-1] = "Compl"

        if i > 0 and sequence[i] == True and sequence[i-1] == False:
            if i == len(sequence) - 1:
                labels[-2] = "Resch"
                labels[-1] = "Cancel"
            else:
                labels[-1] = "Resch"

    return labels

# Test cases
print(process_sequence([True, True, False]))  # Resch Resch Compl
print(process_sequence([True, False, False]))  # Resch Compl Compl
print(process_sequence([False, False]))  # Compl Compl
print(process_sequence([True, True]))  # Resch Cancel
print(process_sequence([True]))  # Cancel
print(process_sequence([False]))  # Compl
print(process_sequence([True, False]))  # Resch Compl
print(process_sequence([False, True]))  # Resch Cancel
# print(process_sequence([False, True, True]))  # Resch Resch Cancel
# print(process_sequence([False, True, True, True]))  # Resch Resch Resch Cancel
# print(process_sequence([False, FalsCreatede, True, True]))  # Compl Resch Resch Cancel
# print(process_sequence([False, False, False, True]))  # Compl Compl Resch Cancel


['Resch', 'Resch', 'Compl']
['Resch', 'Compl', 'Compl']
['Compl', 'Compl']
['Resch', 'Cancel']
['Cancel']
['Compl']
['Resch', 'Compl']
['Resch', 'Cancel']


In [18]:
sorted_sample[['Canceled', 'Start Date & Time', 'Canceled By', 'Cancellation reason', 'Event Type Name', 'Event Created Date & Time']].head(50)

,Canceled,Start Date & Time,Canceled By,Cancellation reason,Event Type Name,Event Created Date & Time
27,True,2023-06-08 20:05:00,Invitee,NaN,Stretching,2023-06-06 09:36:00
15,False,2023-06-08 11:00:00,NaN,NaN,Stretching,2023-06-07 09:19:00
67,True,2023-06-10 11:00:00,Invitee,NaN,Stretching,2023-06-08 14:06:00
69,False,2023-06-12 20:05:00,NaN,NaN,Stretching,2023-06-11 15:38:00
90,False,2023-06-17 11:00:00,NaN,NaN,Stretching,2023-06-11 15:40:00
100,False,2023-06-15 20:05:00,NaN,NaN,Stretching,2023-06-13 12:04:00
131,False,2023-06-19 20:05:00,NaN,NaN,Stretching,2023-06-18 17:04:00
204,False,2023-06-28 19:00:00,NaN,NaN,Stretching,2023-06-26 00:12:00
236,False,2023-07-01 11:00:00,NaN,NaN,Stretching,2023-06-29 10:46:00
275,False,2023-07-03 20:05:00,NaN,NaN,Stretching,2023-07-02 12:11:00


# Extract Compl,Resch,Cancel Metrics from the User profile

In [32]:
from tqdm import tqdm
# TODO filter values if Canceled By = HOST

# set some parameters
vis = True
outliers_group = pd.DataFrame()
users_metrics = pd.DataFrame()

# load user data
for _, user in tqdm(processed_data.groupby('Invitee Email')):
    # user = processed_data[processed_data['Invitee Email'] == 'mansonderek@hotmail.com']

    # Convert data to DateTime format
    user = convert_to_datetime_format(user, ['Start Date & Time', 'End Date & Time', 'Event Created Date & Time'])
    sorted_user_by_event_time = user.sort_values(by='Event Created Date & Time')

    # Group user data by Event Date & Time AND Event Type Name
    filter_fn = lambda x: sorted_user_by_event_time.loc[x]['Start Date & Time'].date()
    grouped_user_data_by_event_time_and_name = sorted_user_by_event_time.groupby([filter_fn, 'Event Type Name'])

    # Iterate by group
    groups = [group for _, group in grouped_user_data_by_event_time_and_name]

    # Process the grouped user data
    for group in groups:
        if (len(group) >= 3) and (group['Canceled'].iloc[-1] == True):
            outliers_group = pd.concat((outliers_group, group))
        current_status = group['Canceled']
        labels = process_sequence(current_status)

        for i in range(len(current_status - 1)):
            curIndex = current_status.index[i]
            user.at[curIndex, 'EventStatus'] = labels[i]
    
    # Count Metrics
    event_type_names = user['Event Type Name'].unique()
    grouped_by_event = user.groupby('Event Type Name')
    for name, group in grouped_by_event:
        compl, resch, cancel = get_metrics_from_grouped_df(group)
        user_metric[f'{name}_compl'] = compl
        user_metric[f'{name}_resch'] = resch
        user_metric[f'{name}_cancel'] = cancel
    
        

    user_metric_df = pd.DataFrame([user_metric])
    user_metric_df['User_ID'] = user['Invitee Email'].iloc[0]
    

    # Add user to users metrics

    users_metrics = pd.concat((users_metrics, user_metric_df))

    user_metric = {
        'Stretching_compl': 0,
        'Stretching_resch': 0,
        'Stretching_cancel': 0,
        'Yoga_compl': 0,
        'Yoga_resch': 0,
        'Yoga_cancel': 0,
        '`Body Ballet_compl': 0,
        '`Body Ballet_resch': 0,
        '`Body Ballet_cancel': 0,
        'Ballet_comp': 0,
        'Ballet_resch': 0,
        'Ballet_cancel': 0,
    }



users_metrics.set_index('User_ID')

if not len(outliers_group):
    print(f"INFO: Here is no outlieres.")
else:
    print(f'WARN: Here is {len(outliers_group)} outlieres.')

if not user['EventStatus'].isna().sum():
    print(f"INFO: The data was processed successfuly.")

if vis:
    user[['EventStatus', 'Canceled', 'Start Date & Time', 'Event Created Date & Time']]
    sorted_user_data_after_processing = user.sort_values(by='Event Created Date & Time')
    (sorted_user_data_after_processing[['Canceled', 'Start Date & Time', 'Canceled By', 'Event Type Name', 'Event Created Date & Time', 'EventStatus']])


  0%|          | 0/661 [00:00<?, ?it/s]

100%|██████████| 661/661 [00:05<00:00, 130.42it/s]

WARN: Here is 14 outlieres.
INFO: The data was processed successfuly.


In [33]:
users_metrics[['']]

,Stretching_compl,Stretching_resch,Stretching_cancel,Yoga_compl,Yoga_resch,Yoga_cancel,`Body Ballet_compl,`Body Ballet_resch,`Body Ballet_cancel,Ballet_comp,Ballet_resch,Ballet_cancel,Ballet_compl,User_ID
0,6,0,1,1,0,0,13,0,0,0,0,1,0.0,0454400@qq.com
0,1,0,0,1,0,0,1,0,0,0,0,0,NaN,7416782@gmail.com
0,29,0,2,0,0,0,0,0,0,0,0,0,NaN,a.b.szczepanska@gmail.com
0,0,0,0,1,0,0,0,0,0,0,0,0,NaN,a.jawaid@nencki.edu.pl
0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,a.lot.of.banjos@gmail.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,0,0,1,0,0,0,0,0,0,0,0,NaN,zpenelolaz@icloud.com
0,0,0,0,5,1,2,0,0,0,0,0,0,NaN,zpenelopaz@icloud.com
0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,zrazhaeva.a@gmail.com
0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,zuziagrabek2403@gmail.com


In [19]:
from tqdm import tqdm
# TODO filter values if Canceled By = HOST

# set some parameters
vis = True
outliers_group = pd.DataFrame()
users_metrics = pd.DataFrame()

# load user data
user = processed_data[processed_data['Invitee Email'] == 'an.birukovaa@gmail.com']

# Convert data to DateTime format
user = convert_to_datetime_format(user, ['Start Date & Time', 'End Date & Time', 'Event Created Date & Time'])
sorted_user_by_event_time = user.sort_values(by='Event Created Date & Time')

# Group user data by Event Date & Time AND Event Type Name
filter_fn = lambda x: sorted_user_by_event_time.loc[x]['Start Date & Time'].date()
grouped_user_data_by_event_time_and_name = sorted_user_by_event_time.groupby([filter_fn, 'Event Type Name'])

# Iterate by group
groups = [group for _, group in grouped_user_data_by_event_time_and_name]

# Process the grouped user data
for group in groups:
    if (len(group) >= 3) and (group['Canceled'].iloc[-1] == True):
        outliers_group = pd.concat((outliers_group, group))
    current_status = group['Canceled']
    labels = process_sequence(current_status)

    for i in range(len(current_status - 1)):
        curIndex = current_status.index[i]
        user.at[curIndex, 'EventStatus'] = labels[i]

# Count Metrics
event_type_names = user['Event Type Name'].unique()
grouped_by_event = user.groupby('Event Type Name')
for name, group in grouped_by_event:
    compl, resch, cancel = get_metrics_from_grouped_df(group)
    user_metric[f'{name}_compl'] = compl
    user_metric[f'{name}_resch'] = resch
    user_metric[f'{name}_cancel'] = cancel

user_metric_df = pd.DataFrame([user_metric])
user_metric_df['User_ID'] = user['Invitee Email'].iloc[0]

# Add user to users metrics

users_metrics = pd.concat((users_metrics, user_metric_df))
users_metrics.set_index('User_ID')

if not len(outliers_group):
    print(f"INFO: Here is no outlieres.")
else:
    print(f'WARN: Here is {len(outliers_group)} outlieres.')

if not user['EventStatus'].isna().sum():
    print(f"INFO: The data was processed successfuly.")

if vis:
    user[['EventStatus', 'Canceled', 'Start Date & Time', 'Event Created Date & Time']]
    sorted_user_data_after_processing = user.sort_values(by='Event Created Date & Time')
    (sorted_user_data_after_processing[['Canceled', 'Start Date & Time', 'Canceled By', 'Event Type Name', 'Event Created Date & Time', 'EventStatus']])


KeyError: 0

In [211]:
user_metric_df

,Stretching_compl,Stretching_resch,Stretching_cancel,Yoga_compl,Yoga_resch,Yoga_cancel,`Body Ballet_compl,`Body Ballet_resch,`Body Ballet_cancel,Ballet_comp,Ballet_resch,Ballet_cancel,Ballet_compl,User_ID
0,6,0,1,0,0,1,13,0,0,0,0,1,0,an.birukovaa@gmail.com


In [20]:
# Count metrics

event_type_names = user['Event Type Name'].unique()
grouped_by_event = user.groupby('Event Type Name')
for name, group in grouped_by_event:
    compl, resch, cancel = get_metrics_from_grouped_df(group)
    user_metric[f'{name}_compl'] = compl
    user_metric[f'{name}_resch'] = resch
    user_metric[f'{name}_cancel'] = cancel

user_metric_df = pd.DataFrame([user_metric])
user_metric_df['User_ID'] = user['Invitee Email']
user_metric_df.set_index('User_ID')


NameError: name 'get_metrics_from_grouped_df' is not defined

In [21]:

user_metric = {
    'Stretching_compl': 0,
    'Stretching_resch': 0,
    'Stretching_cancel': 0,
    'Yoga_compl': 0,
    'Yoga_resch': 0,
    'Yoga_cancel': 0,
    '`Body Ballet_compl': 0,
    '`Body Ballet_resch': 0,
    '`Body Ballet_cancel': 0,
    'Ballet_comp': 0,
    'Ballet_resch': 0,
    'Ballet_cancel': 0,
}


def get_metrics_from_grouped_df(df: pd.DataFrame):
    df = df.copy()
    compl = df[df['EventStatus'] == 'Compl']['EventStatus'].count()
    resch = df[df['EventStatus'] == 'Resch']['EventStatus'].count()
    cancel = df[df['EventStatus'] == 'Cancel']['EventStatus'].count()

    return compl, resch, cancel

In [22]:
def process_sequence(sequence: pd.DataFrame):
    # Initialize an empty list to hold the output labels
    labels = []

    # Loop through the sequence to determine the output labels based on the rules
    for i in range(len(sequence)):
        if sequence.iloc[i].item() == True:
            if i == len(sequence) - 1:  # If it's the last element
                labels.append("Cancel")
            else:
                labels.append("Resch")
        elif sequence.iloc[i] == False:
            if i == 0 or (i > 0 and sequence.iloc[i-1].item() == False):
                labels.append("Compl")
            else:
                labels.append("Compl")
        
        # Special condition for pairs to determine 'Cancel' or 'Compl'
        if i > 0 and sequence.iloc[i] == False and sequence.iloc[i-1] == True:
            if i == len(sequence) - 1:
                labels[-2] = "Resch"
                labels[-1] = "Compl"
            else:
                labels[-1] = "Compl"

        if i > 0 and sequence.iloc[i] == True and sequence.iloc[i-1] == False:
            if i == len(sequence) - 1:
                labels[-2] = "Resch"
                labels[-1] = "Cancel"
            else:
                labels[-1] = "Resch"

    return labels


In [26]:
from tqdm import tqdm
# TODO filter values if Canceled By = HOST

# set some parameters
vis = True
outliers_group = pd.DataFrame()
users_metrics = pd.DataFrame()

# load user data
user = processed_data[processed_data['Invitee Email'] == 'an.birukovaa@gmail.com']

# Convert data to DateTime format
user = convert_to_datetime_format(user, ['Start Date & Time', 'End Date & Time', 'Event Created Date & Time'])
sorted_user_by_event_time = user.sort_values(by='Event Created Date & Time')

# Group user data by Event Date & Time AND Event Type Name
filter_fn = lambda x: sorted_user_by_event_time.loc[x]['Start Date & Time'].date()
grouped_user_data_by_event_time_and_name = sorted_user_by_event_time.groupby([filter_fn, 'Event Type Name'])

# Iterate by group
groups = [group for _, group in grouped_user_data_by_event_time_and_name]

# Process the grouped user data
for group in groups:
    if (len(group) >= 3) and (group['Canceled'].iloc[-1] == True):
        outliers_group = pd.concat((outliers_group, group))
    current_status = group['Canceled']
    labels = process_sequence(current_status)

    for i in range(len(current_status - 1)):
        curIndex = current_status.index[i]
        user.at[curIndex, 'EventStatus'] = labels[i]

# Count Metrics
event_type_names = user['Event Type Name'].unique()
grouped_by_event = user.groupby('Event Type Name')
for name, group in grouped_by_event:
    compl, resch, cancel = get_metrics_from_grouped_df(group)
    user_metric[f'{name}_compl'] = compl
    user_metric[f'{name}_resch'] = resch
    user_metric[f'{name}_cancel'] = cancel

user_metric_df = pd.DataFrame([user_metric])
user_metric_df['User_ID'] = user['Invitee Email'].iloc[0]

# Add user to users metrics

users_metrics = pd.concat((users_metrics, user_metric_df))
users_metrics.set_index('User_ID')

if not len(outliers_group):
    print(f"INFO: Here is no outlieres.")
else:
    print(f'WARN: Here is {len(outliers_group)} outlieres.')

if not user['EventStatus'].isna().sum():
    print(f"INFO: The data was processed successfuly.")

if vis:
    user[['EventStatus', 'Canceled', 'Start Date & Time', 'Event Created Date & Time']]
    sorted_user_data_after_processing = user.sort_values(by='Event Created Date & Time')
    (sorted_user_data_after_processing[['Canceled', 'Start Date & Time', 'Canceled By', 'Event Type Name', 'Event Created Date & Time', 'EventStatus']])


INFO: Here is no outlieres.
INFO: The data was processed successfuly.


In [27]:
user_metric_df

,Stretching_compl,Stretching_resch,Stretching_cancel,Yoga_compl,Yoga_resch,Yoga_cancel,`Body Ballet_compl,`Body Ballet_resch,`Body Ballet_cancel,Ballet_comp,Ballet_resch,Ballet_cancel,User_ID
0,0,0,0,0,0,1,0,0,0,0,0,0,an.birukovaa@gmail.com


In [36]:
users_metrics[users_metrics['User_ID'] == 'an.birukovaa@gmail.com']

,Stretching_compl,Stretching_resch,Stretching_cancel,Yoga_compl,Yoga_resch,Yoga_cancel,`Body Ballet_compl,`Body Ballet_resch,`Body Ballet_cancel,Ballet_comp,Ballet_resch,Ballet_cancel,Ballet_compl,User_ID
0,0,0,0,0,0,1,0,0,0,0,0,0,NaN,an.birukovaa@gmail.com


In [37]:
users_metrics.to_csv('users_metrics_data.csv')